In [1]:
import time
from sympy import *
import math
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Входные параметры

In [2]:
h = 0.09
a1 = 0
a = 60 * h
b = 60 * h
mu_12 = 0.3
mu_21 = 0.3
k = 5 / 6
R_1 = 255 * h
R_2 = 255 * h
k_x = 1 / R_1
k_y = 1 / R_2
E_1 = 2.1 * 10**5
E_2 = 2.1 * 10**5
A = 1
B = 1

N = 4
n = int(math.sqrt(N))

# Объявление функций

In [3]:
x, y, z, q = symbols('x y z, q')
U = Function('U')(x, y)
V = Function('V')(x, y)
W = Function('W')(x, y)
Psi_x = Function('Psi_x')(x, y)
Psi_y = Function('Psi_y')(x, y)

f = Function('f')(z)
f = 6 * (0.25 - z**2 / h**2)

In [4]:
U, V, W, Psi_x, Psi_y = 0, 0, 0, 0, 0
u = IndexedBase('u')
v = IndexedBase('v')
w = IndexedBase('w')
psi_x = IndexedBase('psi_x')
psi_y = IndexedBase('psi_y')
i, j = symbols('i j', cls=Idx)
for i in range(1, n + 1):
  for j in range(1, n + 1):
    U = U + u[i, j] * sin(i * math.pi * x / a) * sin(j * math.pi * y / b)
    V = V + v[i, j] * sin(i * math.pi * x / a) * sin(j * math.pi * y / b)
    W = W + w[i, j] * sin(i * math.pi * x / a) * sin(j * math.pi * y / b)
    Psi_x = Psi_x + psi_x[i, j] * cos(i * math.pi * x / a) * sin(j * math.pi * y / b)
    Psi_y = Psi_y + psi_y[i, j] * sin(i * math.pi * x / a) * cos(j * math.pi * y / b)

In [5]:
teta_1 = - (1 / A * diff(W, x) + k_x * U)
teta_2 = - (1 / B * diff(W, y) + k_y * V)
eps_x = 1 / A * diff(U, x) + 1 / (A * B) * V * diff(A, y) - k_x * W + 0.5 * teta_1**2
eps_y = 1 / B * diff(V, y) + 1 / (A * B) * U * diff(B, x) - k_y * W + 0.5 * teta_2**2
gam_xy = 1 / A * diff(V, x) + 1 / B * diff(U, y) - 1 / (A * B) * U * diff(A, y) - 1 / (A * B) * V * diff(B, x) + teta_1 * teta_2
gam_xz = k * f * (Psi_x - teta_1)
gam_yz = k * f * (Psi_y - teta_2)
khi_1 = 1 / A * diff(Psi_x, x) + 1 / (A * B) * diff(A, y) * Psi_y
khi_2 = 1 / B * diff(Psi_y, y) + 1 / (A * B) * diff(B, x) * Psi_x
khi_12 = 0.5 * (1 / A * diff(Psi_y, x) + 1 / B * diff(Psi_x, y) - 1 / (A * B) * (diff(A, y) * Psi_x + diff(B, x) * Psi_y))
G_12 = E_1 / (2 * (1 + mu_12))
G_13 = G_12
G_23 = G_12
P_x = 0
P_y = 0

In [6]:
N_x = E_1 / (1 - mu_12 * mu_21) * h * (eps_x + mu_21 * eps_y)
N_y = E_2 / (1 - mu_12 * mu_21) * h * (eps_y + mu_12 * eps_y)
N_xy = G_12 * h * gam_xy
N_yx = N_xy
M_x = E_1 / (1 - mu_12 * mu_21) * h**3 / 12 * (khi_1 + mu_21 * khi_2)
M_y = E_2 / (1 - mu_12 * mu_21) * h**3 / 12 * (khi_2 + mu_12 * khi_1)
M_xy = 2 * G_12 * h**3 / 12 * khi_1
M_yx = M_xy
Q_x = G_13 * k * h * (Psi_x - teta_1)
Q_y = G_23 * k * h * (Psi_y - teta_2)

In [7]:
def simpsonRule(expression, variable, startValue, endValue, numOfIntervals = 10):
  # проверки на наличие variable в expression
  # проверки на start-end Values
  step = (endValue - startValue) / numOfIntervals
  result = 0
  result += expression.subs(variable, startValue)
  result += expression.subs(variable, endValue)
  for i in range(1, numOfIntervals):
    result += expression.subs(variable, startValue + step * i) * 2**(i % 2 + 1)

  return result * step / 3

In [8]:
temp_E_s_0 = (N_x * eps_x + N_y * eps_y + 0.5 * (N_xy + N_yx) * gam_xy +
         M_x * khi_1 + M_y * khi_2 + (M_xy + M_yx) * khi_12 +
         Q_x * (Psi_x - teta_1) + Q_y * (Psi_y - teta_2) -
         2 * (q * W + P_x * U + P_y * V)) * A * B
print(temp_E_s_0)

-2*q*(sin(0.581776417331443*x)*sin(0.581776417331443*y)*w[1, 1] + sin(0.581776417331443*x)*sin(1.16355283466289*y)*w[1, 2] + sin(1.16355283466289*x)*sin(0.581776417331443*y)*w[2, 1] + sin(1.16355283466289*x)*sin(1.16355283466289*y)*w[2, 2]) + (-11.4184809909321*sin(0.581776417331443*x)*sin(0.581776417331443*y)*psi_x[1, 1] - 11.4184809909321*sin(0.581776417331443*x)*sin(1.16355283466289*y)*psi_x[1, 2] - 22.8369619818643*sin(1.16355283466289*x)*sin(0.581776417331443*y)*psi_x[2, 1] - 22.8369619818643*sin(1.16355283466289*x)*sin(1.16355283466289*y)*psi_x[2, 2])*(0.290888208665722*cos(0.581776417331443*x)*cos(0.581776417331443*y)*psi_x[1, 1] + 0.290888208665722*cos(0.581776417331443*x)*cos(0.581776417331443*y)*psi_y[1, 1] + 0.581776417331443*cos(0.581776417331443*x)*cos(1.16355283466289*y)*psi_x[1, 2] + 0.290888208665722*cos(0.581776417331443*x)*cos(1.16355283466289*y)*psi_y[1, 2] + 0.290888208665722*cos(1.16355283466289*x)*cos(0.581776417331443*y)*psi_x[2, 1] + 0.581776417331443*cos(1.1635

In [12]:
E_s_0 = 0.5 * simpsonRule(simpsonRule(temp_E_s_0, x, a1, a), y, 0, b)

In [ ]:
X_u, X_v, X_w, X_psix, X_psiy = [], [], [], [], []
for i in range(1, n + 1):
  for j in range(1, n + 1):
    X_u.append(u[i, j])
    X_v.append(v[i, j])
    X_w.append(w[i, j])
    X_psix.append(psi_x[i, j])
    X_psiy.append(psi_y[i, j])
X = X_u + X_v + X_w + X_psix + X_psiy
print(X)

[u[1, 1], u[1, 2], u[2, 1], u[2, 2], v[1, 1], v[1, 2], v[2, 1], v[2, 2], w[1, 1], w[1, 2], w[2, 1], w[2, 2], psi_x[1, 1], psi_x[1, 2], psi_x[2, 1], psi_x[2, 2], psi_y[1, 1], psi_y[1, 2], psi_y[2, 1], psi_y[2, 2]]


In [ ]:
Xn_1 = np.zeros(5 * n**2)
Xn_1_expr = {elem : 0 for _, elem in enumerate(X)}
print(Xn_1_expr)

{u[1, 1]: 0, u[1, 2]: 0, u[2, 1]: 0, u[2, 2]: 0, v[1, 1]: 0, v[1, 2]: 0, v[2, 1]: 0, v[2, 2]: 0, w[1, 1]: 0, w[1, 2]: 0, w[2, 1]: 0, w[2, 2]: 0, psi_x[1, 1]: 0, psi_x[1, 2]: 0, psi_x[2, 1]: 0, psi_x[2, 2]: 0, psi_y[1, 1]: 0, psi_y[1, 2]: 0, psi_y[2, 1]: 0, psi_y[2, 2]: 0}


In [ ]:
eps = 0.005
# hessian = np.zeros((5 * n**2, 5 * n**2))
# gradient = np.zeros(5 * n**2)
gradient = []
hessian = []
for i in range(0, 5 * n**2):
  print(i)
  gradient.append(diff(E_s_0, X[i]))
  hessian.append([diff(gradient[i], X[j]) for j in range(0, 5 * n**2)])
  # for j in range(0, 5 * n**2):
  #   hessian[i][j] = diff(gradient[i], X[j])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
k = 0
X_q = []
for qq in [i * 0.01 for i in range(0, 400)]:
  k += 1
  iter = 0
  while True:
    Xn = Xn_1 - np.matmul(np.linalg.inv(
        np.array([[hessian[i][j].subs(q, qq).subs(Xn_1_expr) for i in range(0, 5 * n**2)]
                  for j in range(0, 5 * n**2)]),
        np.array([gradient[i].subs(q, qq).subs(Xn_1_expr) for i in range(0, 5 * n**2)])))
    subst_X = Xn - Xn_1
    if np.linalg.norm(subst_X) < eps:
      break
    else:
      Xn_1 = np.copy(Xn)
      for I, elem in enumerate(X):
        Xn_1_expr[elem] = Xn_1[i]
    if iter > 10000:
      break
  X_q.append(Xn)

np.save("/content/drive/MyDrive/Colab Notebooks/shellSolves/Упрощенная модель/%s.npy" % "X_q", np.array(X_q))

KeyboardInterrupt: 